# Does resampling experiment help with predicting CtD sentences?

In [1]:
from pathlib import Path
import warnings

import numpy as np
import pandas as pd
import plydata as ply
from sqlalchemy import create_engine

from snorkel.labeling.analysis import LFAnalysis
from snorkeling_helper.generative_model_helper import (
    sample_lfs,
    run_generative_label_function_sampler,
)

warnings.filterwarnings("ignore")

In [2]:
username = "danich1"
password = "snorkel"
dbname = "pubmed_central_db"
database_str = (
    f"postgresql+psycopg2://{username}:{password}@/{dbname}?host=/var/run/postgresql"
)
conn = create_engine(database_str)

## Load the data

In [3]:
label_candidates_dir = Path("../label_candidates/output")
notebook_output_dir = Path("../generative_model_training/output/CtD")

In [4]:
L_abstracts = pd.read_csv(
    str(label_candidates_dir / Path("cd_abstract_train_candidates_resampling.tsv")),
    sep="\t",
)

print(L_abstracts.shape)
L_abstracts.head().T

(971820, 103)


,0,1,2,3,4
LF_HETNET_PHARMACOTHERAPYDB,-1,-1,-1,-1,-1
LF_HETNET_CD_ABSENT,0,0,0,0,0
LF_CD_CHECK_DISEASE_TAG,0,0,0,0,-1
LF_DG_IS_BIOMARKER,-1,-1,-1,-1,-1
LF_DaG_ASSOCIATION,-1,-1,-1,-1,-1
...,...,...,...,...,...
LF_GG_BICLUSTER_INCREASES_EXPRESSION,-1,-1,-1,-1,-1
LF_GG_BICLUSTER_SIGNALING,-1,-1,-1,-1,-1
LF_GG_BICLUSTER_IDENTICAL_PROTEIN,-1,-1,-1,-1,-1
LF_GG_BICLUSTER_CELL_PRODUCTION,-1,0,-1,-1,-1


In [5]:
L_dev = pd.read_csv(
    str(label_candidates_dir / Path("cd_dev_test_candidates_resampling.tsv")), sep="\t"
) >> ply.query("split==10")
print(L_dev.shape)
L_dev.head().T

(500, 105)


,1,2,3,4,6
LF_HETNET_PHARMACOTHERAPYDB,-1.0,-1.0,-1.0,1.0,1.0
LF_HETNET_CD_ABSENT,0.0,0.0,0.0,-1.0,-1.0
LF_CD_CHECK_DISEASE_TAG,-1.0,0.0,0.0,-1.0,-1.0
LF_DG_IS_BIOMARKER,-1.0,-1.0,-1.0,-1.0,-1.0
LF_DaG_ASSOCIATION,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...
LF_GG_BICLUSTER_IDENTICAL_PROTEIN,-1.0,-1.0,-1.0,-1.0,-1.0
LF_GG_BICLUSTER_CELL_PRODUCTION,-1.0,-1.0,-1.0,-1.0,-1.0
split,10.0,10.0,10.0,10.0,10.0
document_id,162660.0,146415.0,213795.0,33875.0,15752.0


In [6]:
L_test = pd.read_csv(
    str(label_candidates_dir / Path("cd_dev_test_candidates_resampling.tsv")), sep="\t"
) >> ply.query("split==11")
print(L_test.shape)
L_test.head().T

(500, 105)


,0,5,8,9,17
LF_HETNET_PHARMACOTHERAPYDB,-1.0,-1.0,-1.0,-1.0,-1.0
LF_HETNET_CD_ABSENT,0.0,0.0,0.0,0.0,0.0
LF_CD_CHECK_DISEASE_TAG,0.0,0.0,-1.0,0.0,0.0
LF_DG_IS_BIOMARKER,-1.0,-1.0,-1.0,-1.0,-1.0
LF_DaG_ASSOCIATION,-1.0,-1.0,-1.0,1.0,-1.0
...,...,...,...,...,...
LF_GG_BICLUSTER_IDENTICAL_PROTEIN,-1.0,-1.0,-1.0,-1.0,-1.0
LF_GG_BICLUSTER_CELL_PRODUCTION,-1.0,-1.0,-1.0,-1.0,-1.0
split,11.0,11.0,11.0,11.0,11.0
document_id,207885.0,291052.0,165878.0,186450.0,212095.0


## Resort the Candidates Based on Abstract

In [7]:
# Grab the document ids for resampling
sql = """
select cd_candidates.sentence_id, document_id, cd_candidates.candidate_id
from sentence
inner join (
  select candidate.candidate_id, compound_disease.sentence_id from compound_disease
  inner join candidate on candidate.candidate_id=compound_disease.candidate_id
  ) as cd_candidates
on sentence.sentence_id = cd_candidates.sentence_id
"""
candidate_doc_df = pd.read_sql(sql, database_str)
candidate_doc_df.head()

,sentence_id,document_id,candidate_id
0,578151345,24670128,8160
1,568187139,27952167,9876
2,561956917,8224816,20087
3,244310297,28588380,45149
4,593065266,15261556,89224


In [8]:
dev_test_ids = (
    L_dev >> ply.select("document_id") >> ply.distinct() >> ply.pull("document_id")
)

filtered_candidate_id = (
    candidate_doc_df
    >> ply.query(f"document_id in {list(dev_test_ids)}")
    >> ply.pull("candidate_id")
)

In [9]:
sorted_train_df = pd.read_csv(
    str(notebook_output_dir / Path("ctd_dataset_mapper.tsv")), sep="\t"
)
sorted_train_df.head()

,document_id,dataset
0,24670128,train
1,27952167,train
2,8224816,train
3,28588380,tune
4,15261556,train


In [10]:
trained_documents = (
    sorted_train_df
    >> ply.inner_join(candidate_doc_df, on="document_id")
    >> ply.query("dataset=='train'")
    >> ply.pull("candidate_id")
)

In [11]:
filtered_L_abstracts = L_abstracts >> ply.query(
    f"candidate_id in {list(trained_documents)}"
)
print(filtered_L_abstracts.shape)
filtered_L_abstracts.head()

(680470, 103)


,LF_HETNET_PHARMACOTHERAPYDB,LF_HETNET_CD_ABSENT,LF_CD_CHECK_DISEASE_TAG,LF_DG_IS_BIOMARKER,LF_DaG_ASSOCIATION,LF_DaG_WEAK_ASSOCIATION,LF_DaG_NO_ASSOCIATION,LF_DaG_CELLULAR_ACTIVITY,LF_DaG_DISEASE_SAMPLE,LF_DG_METHOD_DESC,...,LF_GG_NO_VERB,LF_GG_BICLUSTER_BINDING,LF_GG_BICLUSTER_ENHANCES,LF_GG_BICLUSTER_ACTIVATES,LF_GG_BICLUSTER_AFFECTS_EXPRESSION,LF_GG_BICLUSTER_INCREASES_EXPRESSION,LF_GG_BICLUSTER_SIGNALING,LF_GG_BICLUSTER_IDENTICAL_PROTEIN,LF_GG_BICLUSTER_CELL_PRODUCTION,candidate_id
0,-1,0,0,-1,-1,-1,-1,-1,1,-1,...,-1,-1,-1,1,-1,-1,-1,-1,-1,52
3,-1,0,0,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,408
4,-1,0,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,1,-1,-1,-1,-1,-1,409
5,-1,0,0,-1,-1,-1,-1,-1,1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,410
6,-1,0,-1,-1,-1,-1,-1,-1,1,0,...,-1,-1,-1,1,-1,-1,-1,-1,-1,413


## Construct the Grid Search

In [12]:
# Global Grid
epochs_grid = [500]
l2_param_grid = np.linspace(0.01, 5, num=5)
lr_grid = [1e-2]
grid = list(
    zip(epochs_grid * len(l2_param_grid), l2_param_grid, lr_grid * len(l2_param_grid))
)

# Abstracts

In [13]:
analysis_module = LFAnalysis(
    filtered_L_abstracts >> ply.select("candidate_id", drop=True)
)

abstract_lf_summary = analysis_module.lf_summary()
abstract_lf_summary.index = (
    filtered_L_abstracts >> ply.select("candidate_id", drop=True)
).columns.tolist()

abstract_lf_summary

,Polarity,Coverage,Overlaps,Conflicts
LF_HETNET_PHARMACOTHERAPYDB,[1],0.263468,0.263468,0.263468
LF_HETNET_CD_ABSENT,[0],0.736532,0.736532,0.736532
LF_CD_CHECK_DISEASE_TAG,[0],0.736955,0.736955,0.736955
LF_DG_IS_BIOMARKER,[1],0.070260,0.070260,0.070260
LF_DaG_ASSOCIATION,[1],0.091335,0.091335,0.091335
...,...,...,...,...
LF_GG_BICLUSTER_AFFECTS_EXPRESSION,[1],0.014528,0.014528,0.014528
LF_GG_BICLUSTER_INCREASES_EXPRESSION,[0],0.033060,0.033060,0.033060
LF_GG_BICLUSTER_SIGNALING,[1],0.033065,0.033065,0.033065
LF_GG_BICLUSTER_IDENTICAL_PROTEIN,[0],0.019811,0.019811,0.019811


# Set up fields for resampling

In [14]:
lf_columns_base = list(L_abstracts.columns[0:2])
candidate_id_field = list(L_abstracts.columns[-1:])
dev_column_base = ["split", "curated_ctd", "document_id"]
data_columns = []

# Abstracts - All Label Functions

## CtD

In [15]:
ctd_start = 2
ctd_end = 102

# Spaced out number of sampels including total
size_of_samples = [1, 33, 65, 97, ctd_end - ctd_start]
number_of_samples = 50
ctd_lf_range = range(ctd_start, ctd_end)

In [16]:
sampled_lfs_dict = {
    sample_size: (
        sample_lfs(
            list(ctd_lf_range),
            len(list(ctd_lf_range)),
            sample_size,
            number_of_samples,
            random_state=100,
        )
    )
    for sample_size in size_of_samples
}

In [17]:
data_columns += run_generative_label_function_sampler(
    filtered_L_abstracts,
    L_dev,
    L_test,
    sampled_lfs_dict,
    lf_columns_base=lf_columns_base,
    grid_param=grid,
    marginals_df_file="",
    curated_label="curated_ctd",
    entity_label="ALL",
    data_source="abstract",
)

100%|██████████| 50/50 [26:27<00:00, 31.75s/it]


# Write Performance to File

In [18]:
performance_df = pd.DataFrame.from_records(data_columns)
performance_df

,lf_num,auroc,aupr,bce_loss,sampled_lf_name,label_source,data_source,model,epochs,l2_param,lr_param
0,1,0.686688,0.507890,2.792143,"LF_HETNET_PHARMACOTHERAPYDB,LF_HETNET_CD_ABSEN...",ALL,abstract,tune,500,0.01,0.01
1,1,0.504533,0.328420,0.874426,"LF_HETNET_PHARMACOTHERAPYDB,LF_HETNET_CD_ABSEN...",ALL,abstract,test,500,0.01,0.01
2,1,0.690878,0.522644,2.790183,"LF_HETNET_PHARMACOTHERAPYDB,LF_HETNET_CD_ABSEN...",ALL,abstract,tune,500,0.01,0.01
3,1,0.528063,0.339871,0.872649,"LF_HETNET_PHARMACOTHERAPYDB,LF_HETNET_CD_ABSEN...",ALL,abstract,test,500,0.01,0.01
4,1,0.707070,0.526432,2.790027,"LF_HETNET_PHARMACOTHERAPYDB,LF_HETNET_CD_ABSEN...",ALL,abstract,tune,500,0.01,0.01
...,...,...,...,...,...,...,...,...,...,...,...
495,100,0.639866,0.366371,5.135602,"LF_HETNET_PHARMACOTHERAPYDB,LF_HETNET_CD_ABSEN...",ALL,abstract,test,500,5.00,0.01
496,100,0.708114,0.330428,3.440006,"LF_HETNET_PHARMACOTHERAPYDB,LF_HETNET_CD_ABSEN...",ALL,abstract,tune,500,5.00,0.01
497,100,0.639866,0.366371,5.135602,"LF_HETNET_PHARMACOTHERAPYDB,LF_HETNET_CD_ABSEN...",ALL,abstract,test,500,5.00,0.01
498,100,0.708114,0.330428,3.440006,"LF_HETNET_PHARMACOTHERAPYDB,LF_HETNET_CD_ABSEN...",ALL,abstract,tune,500,5.00,0.01


In [19]:
(
    performance_df
    >> ply.call(
        "to_csv",
        str(Path("output") / Path("ALL_CtD_performance.tsv")),
        index=False,
        sep="\t",
    )
)